# Cargo Holds: Clean or Dirty
___

#### Background

Owning and operating a dry bulk vessel is challenging.  Earning a profit on any voyage is not a given even with careful calculations, knowledge, and a solid strategy.  Not only only are these profit margins getting thinner but they are absorbing more and more risk to earn them.  One of those risks is fixing a cargo that the vessel's cargo holds might not be ready for on time.  This leads to costly delays, tens of thousands of dollars in time, additional cleaning costs, and damage to a carriers reputation with a charterer.
___

#### Problem Statement

Vessel operators have to decide a vessel's next cargo well ahead of knowing the condition her holds will be in when the vessel arrives to load the cargo.  The operator may have some knowledge to this problem, such as: the vessel's cargo histroy, overall condition of her holds coatings from the time of hire, and possibly the crew's experience and capability preparing the vessel's holds.  However, the determination of the suitability of her holds is left to an inspector's review of her holds before loading that the vessel operator does not have knowledge of in advance.
___

#### Solution

That is why we have created this model.  With this deep learning CNN model, we have trained it on thousands of images of clean and dirty cargo holds against pictures of vessels that have passed inspection.  With this tool the vessel operator can quickly determine the likliehood, not a guarantee, that the vessel's holds will be accepted for the intended cargo.
___

#### Evidence

This project lays out the steps and strategy taken to produce a model of distinguishing between a clean and dirty cargo hold.  The model has been measured for accuracy with the goal of reducing false positives.  In the problem of classifying a hold as clean or dirty, clean is a positive outcome and dirty is a negative outcome.  Therefore a false positive, an instance where the model incorrectly predicts the holds are clean, are limited. 
___

#### Engage

The model is only as good as the images it is provided.  If the images provided omit trouble spots or are not providing enough detail the results will be misleading.  The tool can be tried by submitting an image to the following link.

___

In [ ]:
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
import os
import shutil

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator, smart_resize

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Rescaling, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve, roc_auc_score, RocCurveDisplay, auc

## Data
___

There were no publicly available datasets found online.  The dataset for this problem had to be collected.  From there it will be explored, pre-processed, and organized before being modeled.
___

![Data Directory](../assets/Data_Directory.jpg)


___
#### Data Collection

The data was provided by Nippon Paint Marine, Three Ds Marine Inc, and Seachios Marine Services.  Three Ds Marine Inc is a cargo hold cleaning company based out of the Columbia River.  Seachios Marine Services is based out of Santos Brazil.  They were chosen, as not only do they do great work, as they are hired to prepare vessels for some of the more challenging cargo cleanliness standards to meet.  

The Columbia River is one of the top grain exporteres in the world (behind the Mississippi River and Parana River) while Santos is the main port for grain exports in Brazil. Vessels need to be clean in order to transport grains. While this cleanliness standard is high, it is not the highest as some cargos need "hospital clean" cargo holds. These cargos are extremely sensitive to contamination, such as soda ash and alumina. The Columbia River is one of the largest exporters of soda ash which is used in the production of glass, detergents, batteries amongst other uses.  While Brazil exports large amounts of alumina used in the production of aluminum, ceramics, plastics, paints, and cosmetics. 

Nippon Marine Paint provided some images of freshly painted cargo holds.  Typically cargo holds that have been freshly painted in dry dock do not hire cleaning gangs.  These images are being added to capture this perspective.

Due to their combination of quality and storage size, jpg images are preferred for this project.
___

In [ ]:
# Collect list of image names
sources_path = '../data/sources/'
sources_directory = {}

for directory in os.listdir(sources_path):
    sources_directory[directory] = [i for i in os.listdir(sources_path + directory)]
    
sources_directory.keys()

In [ ]:
# Create a dataframe to track image files
rows = []
for k, v in sources_directory.items():
    for value in v:
        rows.append([k, value])
sources_df = pd.DataFrame(rows, columns=['source', 'original_name'])
sources_df.head()

In [ ]:
# Collect list of image names
collected_path = '../data/collected'
raw_images = [i for i in os.listdir(collected_path)]
len(raw_images)

In [ ]:
# Count of different types
image_types = []
for image in raw_images:
    file_types = image.rsplit('.', 1)[-1].lower()
    image_types.append(file_types)
image_types_count = pd.Series(image_types).value_counts()
image_types_count

___
#### Data Classification
Images were classified as dirty or clean after visual review.  To track the same, they were updated on the sources_df along with a new name.
___

In [ ]:
# Update dataframe to indicate image class
clean_path = '../data/classified/clean/'
clean_images = [i for i in os.listdir(clean_path)]

dirty_path = '../data/classified/dirty/'
dirty_images = [i for i in os.listdir(dirty_path)]

not_used = []
for image in raw_images:
    if image not in clean_images and image not in dirty_images:
        not_used.append(image)

sources_df['class'] = sources_df['original_name'].apply(
    lambda image: 'clean' if image in clean_images 
    else ('dirty' if image in dirty_images else 'not_used'))

sources_df['class'].value_counts()

In [ ]:
# Add new name, not renaming
sources_df['new_name'] = sources_df['class'] + '_' + sources_df.index.astype(str)
sources_df.sort_values(by='source', inplace=True)
sources_df.head(5)

In [ ]:
# Checked not used files as not impactful for current intention
not_used

___
#### Data Exploration

The initial dataset contained 560 images.  It was straightforward to classify the holds as clean or dirty working with a dataset of this size.  It was also quickly noticeable that this dataset is imbalanced, with 94 clean cargo holds and 466 dirty cargo holds.  In the real world, this data should be perfectly balanced as a cargo hold is either clean or dirty for the purposes of this problem.  The working process for determining cargo hold cleanlieness would start with dirty cargo holds and end with clean holds.

From this initial classification of the dataset, the images will be explored further to get a better understanding viusally of the data, the types of issues that render a cargo hold dirty, and the distribution of the size of images.
___

In [ ]:
# Size of classes in the dataset
num_clean = len([i for i in os.listdir('../data/classified/clean/')])
num_dirty = len([i for i in os.listdir('../data/classified/dirty/')])
num_clean, num_dirty

In [ ]:
# Pie chart with class distribution
fig, ax = plt.subplots()
ax.pie(x=[num_clean, num_dirty], labels=['Clean', 'Dirty'], colors=['lightblue', 'lightgreen'], autopct='%1.1f%%')
ax.set_title('Class Distribution')
plt.show;

In [ ]:
# Review clean cargo hold image
clean1 = load_img('../data/classified/clean/47681173_2008763662504219_1873423245131120640_n.jpg')
clean2 = load_img('../data/classified/clean/48418327_2026951410685444_6927397904709582848_n.jpg')
clean1

In [ ]:
# Review dirty cargo hold image
dirty1 = load_img('../data/classified/dirty/46706053_1983209351726317_6755771672986386432_n.jpg')
dirty2 = load_img('../data/classified/dirty/46491446_1983209548392964_3471892652591415296_n.jpg')
dirty1

In [ ]:
# Comparing clean to dirty with similar color hold coatings

fig, axes = plt.subplots(2, 2, figsize=(12,12))

axes[0, 0].imshow(clean1)
axes[0, 0].set_title('Clean')
axes[0, 0].axis('off')

axes[0, 1].imshow(clean2)
axes[0, 1].set_title('Clean')
axes[0, 1].axis('off')

axes[1, 0].imshow(dirty1)
axes[1, 0].set_title('Dirty')
axes[1, 0].axis('off')

axes[1, 1].imshow(dirty2)
axes[1, 1].set_title('Dirty')
axes[1, 1].axis('off')

plt.show;

In [ ]:
# different types of issues

cargoresidue = load_img('../data/classified/dirty/12c943_1ff40351da324d869a7f3b4c40d1b3e2~mv2.jpg')
cargoresidue2 = load_img('../data/classified/dirty/12c943_2fa04463d9254f2b8baf668e14de169a~mv2.jpg')
rustscale = load_img('../data/classified/dirty/66714988_2345934515453797_4846150941700784128_n.jpg')
tackypaint = load_img('../data/classified/dirty/12c943_e53efdefc0294acaa30e0a394f266436~mv2.jpg')
flakingpaint = load_img('../data/classified/dirty/12c943_efd8bd3332474e328fdbcef5cb7d2f46~mv2.jpg')
staining = load_img('../data/classified/dirty/46675044_1983208488393070_2248909549803143168_n.jpg')

fig, axes = plt.subplots(2, 3, figsize=(12,12))

axes[0, 0].imshow(cargoresidue)
axes[0, 0].set_title('Cement Cargo Residue')
axes[0, 0].axis('off')

axes[0, 1].imshow(cargoresidue2)
axes[0, 1].set_title('Petcoke Cargo Residue')
axes[0, 1].axis('off')

axes[0, 2].imshow(rustscale)
axes[0, 2].set_title('Rust Scale')
axes[0, 2].axis('off')

axes[1, 0].imshow(tackypaint)
axes[1, 0].set_title('Tacky Paint')
axes[1, 0].axis('off')

axes[1, 1].imshow(flakingpaint)
axes[1, 1].set_title('Flaking Paint')
axes[1, 1].axis('off')

axes[1, 2].imshow(staining)
axes[1, 2].set_title('Cargo Staining')
axes[1, 2].axis('off')

plt.show;

In [ ]:
# gather image size ranges
clean_path = '../data/classified/clean/'
dirty_path = '../data/classified/dirty/'

clean_sizes = [load_img(os.path.join(clean_path, i)).size for i in os.listdir(clean_path)]
dirty_sizes = [load_img(os.path.join(dirty_path, i)).size for i in os.listdir(dirty_path)]

max(clean_sizes), min(clean_sizes), max(dirty_sizes), min(dirty_sizes)

In [ ]:
# Review image size distribution
clean_sizes_dist = pd.Series(clean_sizes).value_counts()
dirty_sizes_dist = pd.Series(dirty_sizes).value_counts() 

size_dist = pd.concat([clean_sizes_dist, dirty_sizes_dist], axis=1)
size_dist.columns = ['Clean', 'Dirty']
size_dist.fillna(0, inplace=True)
size_dist['Total'] = size_dist['Clean'] + size_dist['Dirty']
size_dist_sorted = size_dist.sort_values('Total', ascending=False)

# plotting distributions
ax = size_dist_sorted[['Clean', 'Dirty']].plot(figsize=(10,10), kind='bar', stacked=True)
ax.set_title('Distribution of Image Sizes')
ax.set_ylabel('Count')
ax.set_xlabel('Number of Pixels (w x h)')
plt.show();

___
#### Data Pre-processing

This project is working with an imbalanced and small dataset.  There are some preprocessing steps to take to improve the model performance.  These steps can be tailored in the future as the dataset size increases.

* Splitting
* Image Augmentation
* Image Generator

Before creating synthetic images to balance out the dataset, we will hold back 10% for a test set of unaltered images.  This will be based on 10% of the dirty images, the target number for balancing the minority dataset.  This is a relatively large set, 20% of the clean images, but this will eliminate overstating or understating the evaluation of the model.  In time this will not be as significant as te dataset is natually balanced out.
___



In [ ]:
# number of images to hold back for test set
test_target = round(len(dirty_images) / 10, 0)
test_target

In [ ]:
# Holding 10 percent images from each class to serve as a test set.

# get ten random clean images
random.shuffle(clean_images)
test_clean_list = clean_images[:55]

# make a filepath
test_clean = []
for image in test_clean_list:
    test_clean.append(clean_path+image)

# move to test directory
for clean_image in test_clean:
    shutil.move(clean_image, '../data/test/clean/')

# get ten random dirty images
random.shuffle(dirty_images)
test_dirty_list = dirty_images[:55]

# make a filepath
test_dirty = []
for image in test_dirty_list:
    test_dirty.append(dirty_path+image)

# move to test directory
for dirty_image in test_dirty:
    shutil.move(dirty_image, '../data/test/dirty/')

___
#### Image Augmentation
Synthetic images of the clean cargo hold class will be generated to address the class imbalance.  Oversampling the minority class could also work, but synthetic generation is preferred to avoid creating a bias by relying on multiple samples of the same image.  Synthetic images create more variety with synthetic creation, flips, crops, etc to avoid introducing bias to repeated samples.

It should be noted that this approach is not ideal.  The size of this dataset is insufficient to provide confidence in the model's performance.  Only after obtaining more data (roughly 1000 images per class) should the model be considered for production, without creating synthetic images.

In the real world, this dataset should be equally balanced. The vessel should provide pictures of dirty holds and eventually clean holds. A vessel's cargo holds are either dirty or clean.ved.
___



In [ ]:
# generating more images from original dataset
balance_datagen = ImageDataGenerator(
    brightness_range=(0.5, 1.5),
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=[0.4, 0.6],
    channel_shift_range=100,
    horizontal_flip=True,
    fill_mode='nearest',
)

In [ ]:
def create_synthetics(image_list, target, destination_path):
    """Create more images from a given image list and save to destination"""
    # determine how many images to generate per image
    generate_per_image = target / (len(image_list))

    # loop through image list and generate synthetic images
    for image in image_list:
        try:
            x = load_img(image)
            x = img_to_array(x)
            x = x.reshape((1,) + x.shape)
            i = 0
            for batch in balance_datagen.flow(x, batch_size=1, save_to_dir=destination_path, save_prefix='aug', save_format='jpg'):
                i += 1
                if i > generate_per_image:
                    break  
        except FileNotFoundError:
            print(f'{image} was not found as it was moved to the Test dataset. Skipping ...')
            continue

In [ ]:
# Creating more clean images

#clean_class_images = [clean_path + file for file in clean_images]
#clean_augmented = '../data/augmented/clean/'
#clean_target = 375
#create_synthetics(clean_class_images, clean_target, clean_augmented)""

In [ ]:
# Creating more dirty images so model sees some orientation variation

#dirty_class_images = [dirty_path + file for file in dirty_images]
#dirty_augmented = '../data/augmented/dirty/'
#dirty_target = 200
#create_synthetics(dirty_class_images, dirty_target, dirty_augmented)

In [ ]:
num_aug_clean = len([i for i in os.listdir('../data/augmented/clean/')])
num_aug_dirty = len([i for i in os.listdir('../data/augmented/dirty/')])
num_aug_clean, num_aug_dirty

In [ ]:
# Pie chart with class distribution
fig, ax = plt.subplots()
ax.pie(x=[num_aug_clean, num_aug_dirty], labels=['Clean', 'Dirty'], colors=['lightblue', 'lightgreen'], autopct='%1.1f%%')
ax.set_title('Class Distribution')
plt.show;

In [ ]:
# split the remaining images into training and validation datasets

# training & validation clean
try:
    augmented_clean_path = '../data/augmented/clean/'
    augmented_clean_images = [i for i in os.listdir(augmented_clean_path)]
    random.shuffle(augmented_clean_images)
    
    train_clean = []
    augmented_clean_train = augmented_clean_images[:520]
    for act_image in augmented_clean_train:
        train_clean.append(os.path.join(augmented_clean_path + act_image))
    for train_clean_image in train_clean:
        shutil.move(train_clean_image, '../data/train/clean/')
        
    val_clean = []
    augmented_clean_val = augmented_clean_images[520:]
    for acv_image in augmented_clean_val:
        val_clean.append(os.path.join(augmented_clean_path + acv_image))
    for val_clean_image in val_clean:
        shutil.move(val_clean_image, '../data/validate/clean/')
        
    #training & validation dirty
    augmented_dirty_path = '../data/augmented/dirty/'
    augmented_dirty_images = [i for i in os.listdir(augmented_dirty_path)]
    random.shuffle(augmented_dirty_images)
    
    train_dirty = []
    augmented_dirty_train = augmented_dirty_images[:520]
    for adt_image in augmented_dirty_train:
        train_dirty.append(os.path.join(augmented_dirty_path + adt_image))
    for train_dirty_image in train_dirty:
        shutil.move(train_dirty_image, '../data/train/dirty/')
    
    val_dirty = []
    augmented_dirty_val = augmented_dirty_images[520:]
    for adv_image in augmented_dirty_val:
        val_dirty.append(os.path.join(augmented_dirty_path + adv_image))
    for val_dirty_image in val_dirty:
        shutil.move(val_dirty_image, '../data/validate/dirty/')

except FileNotFoundError as e:
    print(f'Image was not found {e} as it was moved to the Test dataset. Skipping ...')

In [ ]:
num_train_clean = len([i for i in os.listdir('../data/train/clean/')])
num_train_dirty = len([i for i in os.listdir('../data/train/dirty/')])

num_val_clean = len([i for i in os.listdir('../data/validate/clean/')])
num_val_dirty = len([i for i in os.listdir('../data/validate/dirty/')])

num_test_clean = len([i for i in os.listdir('../data/test/clean/')])
num_test_dirty = len([i for i in os.listdir('../data/test/dirty/')])

num_train_clean, num_train_dirty, num_val_clean, num_val_dirty, num_test_clean, num_test_dirty

In [49]:
# Update dataframe to indicate train/validate/test split

# Create lists of where each image was assigned
train_clean_path = '../data/train/clean'
train_clean_images = [i for i in os.listdir(train_clean_path)]
train_dirty_path = '../data/train/dirty'
train_dirty_images = [i for i in os.listdir(train_dirty_path)]
validate_clean_path = '../data/validate/clean'
validate_clean_images = [i for i in os.listdir(validate_clean_path)]
validate_dirty_path = '../data/validate/dirty'
validate_dirty_images = [i for i in os.listdir(validate_dirty_path)]
test_clean_path = '../data/test/clean'
test_clean_images = [i for i in os.listdir(test_clean_path)]
test_dirty_path = '../data/test/dirty'
test_dirty_images = [i for i in os.listdir(test_dirty_path)]


# store them in a dictionary and convert to a dataframe
split_dict = {
    'split': 
    ['train_clean_images'] * len(train_clean_images) + 
    ['train_dirty_images'] * len(train_dirty_images) +
    ['validate_clean_images'] * len(validate_clean_images) + 
    ['validate_dirty_images'] * len(validate_dirty_images) +
    ['test_clean_images'] * len(test_clean_images) + 
    ['test_dirty_images'] * len(test_dirty_images),
    'original_name': 
    train_clean_images + 
    train_dirty_images + 
    validate_clean_images + 
    validate_dirty_images +
    test_clean_images + 
    test_dirty_images
}

split_df = pd.DataFrame(split_dict)
split_df = pd.merge(split_df, sources_df, on='original_name', how='left')
split_df

,split,original_name,source,class,new_name
0,train_clean_images,046c7aff-205a-46c5-8412-6b6fac84fb00.jpg,Seachios,clean,clean_6
1,train_clean_images,0a9ce194-f3f2-4efe-9621-c0df37e23e77.jpg,Seachios,clean,clean_9
2,train_clean_images,0b80c8d6-b7ed-4d79-a706-998916e0a382.jpg,Seachios,not_used,not_used_11
3,train_clean_images,1000052030.jpg,Seachios,clean,clean_13
4,train_clean_images,1000052031.jpg,Seachios,clean,clean_14
...,...,...,...,...,...
1255,test_dirty_images,IMG_2141.jpg,Three Ds Marine,not_used,not_used_793
1256,test_dirty_images,IMG_2142.jpg,Three Ds Marine,not_used,not_used_794
1257,test_dirty_images,IMG_2179.jpg,Three Ds Marine,not_used,not_used_806
1258,test_dirty_images,IMG_2182.jpg,Three Ds Marine,not_used,not_used_808


In [ ]:
split_df

In [ ]:
# store them in a dictionary and convert to a dataframe
split_dict = {
    'train_clean': train_clean_images,
    'train_dirty': train_dirty_images,
    'validate_clean': validate_clean_images,
    'validate_dirty': validate_dirty_images,
    'test_clean': test_clean_images,
    'test_dirty': test_dirty_images,
}


In [ ]:
# Update dataframe to indicate train/validate/test split
train_clean_path = '../data/train/clean'
train_clean_images = [i for i in os.listdir(clean_path)]

dirty_path = '../data/classified/dirty/'
dirty_images = [i for i in os.listdir(dirty_path)]

not_used = []
for image in raw_images:
    if image not in clean_images and image not in dirty_images:
        not_used.append(image)

sources_df['class'] = sources_df['original_name'].apply(
    lambda image: 'clean' if image in clean_images 
    else ('dirty' if image in dirty_images else 'not_used'))

sources_df['class'].value_counts()

In [ ]:
# Add new name, not renaming
sources_df['split'] = sources_df['class'] + '_' + sources_df.index.astype(str)
sources_df.sort_values(by='source', inplace=True)
sources_df.head(5)

___
#### Modeling

Now that synthetic data has been created to balance the imbalanced datasets, the model can be constructed and fit.  The dataset is still small, so to increase the size of the training set, the training data will be augmented on the fly during training using tensorflow ImageDataGenerator.  The rotations there will be kept modest as a significant portion of synthetic data was already generated.  Additionally the model will be regularized and normalized (by rescaling, dropout, batchnormalization, l2, learning rate, and early stopping )
___

In [ ]:
# create the image data generator to apply to the transformations to data
# the transformations are realistic as it relates to what could be seen
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    brightness_range=(0.8, 1.2),
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
)

validate_datagen = ImageDataGenerator(
    rescale = 1./255
)

# Loading in image data
model_target_size = (224, 224)
model_batch_size = 32

train_generator = train_datagen.flow_from_directory(
    '../data/train/',
    target_size=model_target_size,
    batch_size=model_batch_size,
    class_mode='binary',
    shuffle=True,
    seed=27,
)

validation_generator = validate_datagen.flow_from_directory(
    '../data/validate/',
    target_size=model_target_size,
    batch_size=model_batch_size,
    class_mode='binary',
    shuffle=True,
    seed=27,
)

# model for image classification
model = Sequential()
model.add(Input((224, 224, 3)))

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.0001)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

# compile model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='bce', metrics=['accuracy', 'AUC', Precision(), Recall()])

# architecture
model.summary()

In [ ]:
# model for image classification
model = Sequential()
model.add(Input((224, 224, 3)))

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.0001)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

# compile model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='bce', metrics=['accuracy', 'AUC', Precision(), Recall()])

# architecture
model.summary()

In [ ]:
# add callbacks
checkpoint_path='../assets/saved_models/'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1, save_weights_only=False)
earlystopping = EarlyStopping(restore_best_weights=True, monitor='val_accuracy', min_delta=0, patience=15, verbose=1, mode='auto')

# fit model
history = model.fit(train_generator, epochs=25, validation_data=validation_generator, callbacks=[earlystopping, checkpoint])

In [ ]:
# plotting the training and validation metrics over epochs
plt.figure(figsize=(10,10))
plt.plot(history.history['loss'], label='Training Loss', )
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(history.history['recall_16'], label='Training Recall')
plt.plot(history.history['val_recall_16'], label='Validation Recall')
plt.title('Evaluation over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Metrics')
plt.legend()
plt.show()

In [ ]:
model.save('../assets/saved_models/good_model')

In [ ]:
# load best model for comparison
best_model = load_model('../assets/saved_models/best_model/saved_model.pb')

___
#### Predictions
Let's see how the model does on the test set.  The test set will need to be resaled.
___


In [ ]:
# feed the test dataset to the model to generate predictions
test_datagen = ImageDataGenerator(
    rescale=1./255 # This is a necessary preprocessing step
)
test_generator = test_datagen.flow_from_directory(
    '../data/test/',
    target_size=model_target_size,
    batch_size=model_batch_size,
    class_mode=None,
    shuffle=False
)

print(test_generator.class_indices)

In [ ]:
# make predictions
predictions = model.predict(test_generator, verbose=1)
predicted_class = (predictions > 0.5).astype(int)
class_labels = test_generator.classes
image_names = test_generator.filenames

# print image name and prediction
for image, pred_class in enumerate(predicted_class):
    print(f'Image {image_names[image]} predicted as class: {pred_class}')

In [ ]:
# evaluation metrics
precision = precision_score(class_labels, predicted_class)
recall = recall_score(class_labels, predicted_class)
f1_result = f1_score(class_labels, predicted_class)
confusing_matrix = confusion_matrix(class_labels, predicted_class)

print(f'Precision score: {round(precision, 2)}.')
print(f'Recall score: {round(recall, 2)}.')
print(f'F1 score: {round(f1_result, 2)}.')

cmd = ConfusionMatrixDisplay(confusion_matrix=confusing_matrix, display_labels=test_generator.class_indices)
cmd.plot();

In [ ]:
# Check the ROC Curve - will help us see how well the model is defining classes
fpr, tpr, thresholds = roc_curve(class_labels, predictions)
roc_auc = auc(fpr, tpr)
roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc)
roc_display.plot()
plt.legend()
plt.show;

In [ ]:
# generate the roc_auc_score
roc_auc_score = round(roc_auc_score(class_labels, predictions), 4)
roc_auc_score

In [ ]:
# list of false negatives
false_negatives = []
print('False Negatives')
for image, pred_class in enumerate(predicted_class):
    if pred_class == 0:
        if pred_class != class_labels[image]:
            false_negatives.append(image_names[image])
            print(f'Image {image_names[image]} predicted class "clean" but labeled "dirty".')

In [ ]:
false_negatives

In [ ]:
sources_df.head()